In [1]:
print("Installing required packages...")
!pip install -q transformers torch sentence-transformers faiss-cpu flask pyngrok accelerate bitsandbytes

print("\nAll packages installed successfully!")


Installing required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 9.9 MB/s eta 0:00:00

All packages installed successfully!


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import json
import time
import yaml

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

Libraries imported successfully!
PyTorch version: 2.8.0+cu126
CUDA available: True
CUDA device: Tesla T4


In [3]:
# All 18 Shoplite documents embedded directly in notebook
KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "Shoplite User Registration Process",
        "content": """To sign up to Shoplite, you need to visit the signup page at shoplite. com/register and submit some basic information such as valid e-mail address, safe password (min 8 characters containing at least one upper case letter,one lower case letter,one number and one special character) and profile information like full name and mobile phone number. The email verification process is mandatory and must be completed within 24 hours of registration to activate the account.

Once the basic information are filled, users can select one from between two account types. Buyer profiles are totally free and require email validation only. They provide instant access to shop the products. However, seller accounts need more business verifications such as business name or tax ID number and business address.

The verification process for seller accounts typically takes 2-3 business days as our team reviews submitted documents to ensure compliance with platform standards. Once approved, sellers gain access to inventory management tools, analytics dashboards, and payment processing systems. Users can upgrade from buyer to seller accounts at any time by completing the business verification process through their account settings.

All new users must agree to our Terms of Service and Privacy Policy during registration. Password recovery options can be set up immediately, including security questions and backup email addresses for enhanced account security.
"""
    },
    {
        "id": "doc2",
        "title": "Shoplite Shopping Cart Features",
        "content": """The Shoplite shopping cart is designed for flexibility and convenience, allowing users to add multiple items from different sellers in a single shopping session. Each item added to the cart displays the seller name, product image, price, selected quantity, and any applicable variants such as size or color. The cart automatically calculates subtotals, applicable taxes, and estimated shipping costs based on the delivery address.

Cart contents are preserved across sessions for logged-in users, meaning items remain in the cart even after closing the browser or switching devices. Guest users can also use the shopping cart, but their selections are stored temporarily using browser cookies and will be lost after 7 days of inactivity or when cookies are cleared.

Cart interface has quantity adjustment handles where users could add, subtract or remove items. A “Save for Later” feature lets you lift items out of the active cart without erasing them, a handy way to compare options or wait for sales. Those saved items appear in their own section and can be moved back to the cart all at once with one click.

Promotional codes or discount coupons can be added in the cart before you checkout. The system validates codes in real-time and displays the discounted total immediately. Users are also able to view specific shipping methods and costs prior to placing an order, which allows them to make informed choices about preferred delivery."""
    },
    {
        "id": "doc3",
        "title": "Shoplite Product Search and Filtering",
        "content": """Shoplite's search engine allows to easily find product using multiple search method and several filters. The search bar accepts: product names,brands, keywords or an item number (including sku) using intelligent search algorithms to help you find the perfect fit as you type.
Search results are presented with product thumbnails, prices, ratings and seller information so you can easily find the right product.

The filtering system has multiple criteria to help you narrow your search and find the product you are looking for! You can filter by price range (using a sliding scale), filter out select brands among the checkbox list, see product conditions (new, refurbished, used) and sort by customer ratings. More filters available include seller location, shipping options (free delivery, expedited delivery), and product availability (in stock, pre-order).

If you don't have something specific in mind, category browsing is available from the homepage. Products are sorted by category and more to help make your selection process easier. You’ll find subcategories within those for further specific browsing. For instance Electronics have sub categories such as Smartphones, Laptops, Cameras and Audio equipment.

Sorting options allow users to sort results by relevance, price (low to high and high to low), customer ratings, new arrivals or best sellers. Recent searches are stored, and suggestions provided to help you quickly access them. Users can also save searches to receive alerts when matching new products are listed, or when prices drop on items they are interested in buying."""
    },
     {
        "id": "doc4",
        "title": "Shoplite Payment Methods and Security",
        "content": """Shoplite supports multiple methods of payment to suit different customer preferences. Major credit and debit cards (Visa, Mastercard...) are handled by our secure payment processor with PCI DSS certification. Digital wallets like PayPal, Apple Pay and Google Pay provide an instant checkout for users who have a wallet set up with their payment details.

Bank transfer and direct debit payment options are available to make bulk purchases, but these require 2-3 days to clear payments before orders are processed. Shoplite also enables installments through partners like Klarna and Affirm on purchases over $100 with others, allowing customers to pay monthly over time with different interest rates based on credit approval.

All payment details transmitted between your browser and our website are encrypted using 256bit SSL to ensure security of sensitive information. We do not retain full credit card numbers, which have been tokenized with security tokens. Our payment processing partners are certified as adhering to the highest security and compliance standards, and they are audited annually for compliance with these high standards.

For added security, we deploy fraud-detection systems designed to watch over transactions for unusual activity patterns. All purchases can be protected by two-factor authentication, which ensures users must confirm a purchase via email or SMS before it goes through. Payment details are not disclosed to sellers; it’s an escrow system where shoplite processes payment and disburses the sum to the seller upon successful order completion. Customers will also receive a detailed payment receipt via email upon completion and can access their entire payment history from the account dashboard."""
    },
     {
        "id": "doc5",
        "title": "Shoplite Order Tracking and Delivery",
        "content": """Once an order is placed, customers receive a confirmation email with a unique order number and estimated delivery time. Order tracking begins as soon as the seller ships the item and uploads tracking information to the Shoplite system. Customers can monitor their orders through the "My Orders" section in their account dashboard or by entering their order number on the tracking page.

The tracking service shows real-time updates of order status including all post-purchase steps such as: order confirmation, payment verification, seller preparation, item shipment dispatch and finally delivery completion. Every status update is accompanied by an indication of such, along with the corresponding date and place. Customers receive automatic email (and optional SMS) notifications at important purchase stages, e.g. when your product is dispatched or out for delivery to the recipient.

Delivery timeframes vary based on shipping method selection. Standard shipping typically takes 5-7 business days, expedited shipping takes 2-3 business days, and express overnight shipping delivers within 1 business day for orders placed before 2 PM. International orders have extended delivery windows of 10-21 business days depending on destination country and customs processing.

Shoplite works with most popular carriers, such as FedEx, UPS, USPS and DHL. To track packages, customers can view the carrier that is shipping their order and simply click on the link to the carrier’s tracking system. For valuable shipments, a signature may be required upon receipt. In cases of unsuccessful delivery, delivery services usually leave a notice with specific instruction for package pick-up at nearby locations or rescheduling the delivery."""
    },
     {
        "id": "doc6",
        "title": "Shoplite Return and Refund Policies",
        "content": """Shoplite has a customer-friendly return policy allowing returns within 30 days of delivery for most products. Items must be in original condition with all tags, labels, and packaging intact. The product should be unused, unworn, and in the same condition as received. Digital products, personalized items, intimate apparel, and perishable goods are generally non-returnable unless defective.

To begin the return process, customers will need to sign into their account and go to "My Orders", select the order containing the item they want to return, and click on "Request Return". The system walks users through choosing return reasons and uploading photos if the item is damaged or defective. Return requests are reviewed within 24 to 48h. Once the return request is approved, customers get a prepaid return shipping label in email.

The cost of the return shipping will be based on the reason for the return. Shoplite will cover the cost of return shipping if the item is defective, damaged (through no fault of your own), or incorrect. For returns due to the change of mind or sizing issue, return shipping cost is the responsibility of the customer which gets deduced from the refund amount. Items need to be returned no more than 14 days from date of approval in order to qualify.

Refunds are processed within 5-7 business days after the returned item is received and inspected at our facility. The refund amount is credited to the original payment method used for purchase. For store credit option, customers receive an additional 10% bonus credit that can be used for future purchases. Partial refunds may apply if items are returned without original packaging or show signs of use beyond inspection."""
    },
     {
        "id": "doc7",
        "title": "Shoplite Product Reviews and Ratings",
        "content": """Shoplite’s reviews enable verified buyers – people who have purchased the product in question – to feedback on their purchase, assisting other shoppers make informed choices about what they buy, this helps to reduce fake reviews and maintain the authenticity factor. The review period begins 3 days following delivery, and lasts for 90 days after purchase.

Reviews are star rated (1-5 stars) with the option of leaving a written review. Customers can also post pictures and videos of the product in use, up to 5 per submission, that give other shoppers important visual context. The review form itself contains questions relating to, for example, product quality, description accuracy and value for money as well as the behaviour of seller to help cover most aspects in substantive customer feedback.

All submitted reviews undergo moderation to ensure they meet community guidelines. Reviews containing profanity, personal attacks, competitor promotions, or content unrelated to the product are rejected. The moderation process typically takes 24-48 hours, after which approved reviews appear on the product page. Sellers can respond to reviews to address concerns or thank customers for positive feedback.

The review system calculates the average of all star ratings specified on the product detail page. Reviews can be sorted by helpfuless (based on votes by other users), most recent or highest/lowest ratings. There are also verified purchase badges next to reviews from verified buyers, so it feels a bit more credible. Our customers can receive "Top Reviewer" awards and get their reviews published on the corner of each page for the entire product they own. The system also identifies potentially biased reviews from accounts with suspicious activity patterns."""
    },
     {
        "id": "doc8",
        "title": "Shoplite Seller Account Setup and Management",
        "content": """Becoming a Shoplite seller requires completing a comprehensive registration and verification process. Prospective sellers must first create a basic Shoplite account, then navigate to the Seller Central portal to begin the business application. The application requires business information including legal business name, tax identification number (EIN or SSN for sole proprietors), business address, and bank account details for payment deposits.

Business verification is mandatory and includes document submission such as business licenses, tax registration certificates, and government-issued ID. The verification team reviews applications within 2-3 business days, checking for compliance with legal requirements and platform policies. Some sellers may be asked to provide additional documentation during this process. Once approved, sellers gain immediate access to the Seller Dashboard with full functionality.

The Seller Dashboard provides comprehensive tools for managing inventory, processing orders, communicating with customers, and analyzing sales performance. Sellers can create product listings with descriptions, images, pricing, and inventory quantities. The bulk upload feature allows adding multiple products simultaneously via CSV files, significantly reducing setup time for large catalogs.

Seller accounts include different subscription tiers. The Basic tier is free but includes a per-transaction fee of 12% and limited listing features. The Professional tier costs $39.99 monthly, reduces transaction fees to 8%, and unlocks advanced analytics, promotional tools, and priority customer support. Enterprise sellers with high volume can negotiate custom terms. All sellers must maintain performance metrics including order fulfillment rate above 95%, response time under 24 hours, and customer satisfaction rating above 4.0 stars to remain in good standing.
"""
    },
     {
        "id": "doc9",
        "title": "Shoplite Inventory Management for Sellers",
        "content": """Effective inventory management is crucial for seller success on Shoplite. The inventory management system allows sellers to track stock levels, set low-stock alerts, and automate inventory updates across multiple sales channels. Sellers can manage inventory from the Seller Dashboard under the "Inventory" tab, where all products are listed with current stock quantities, SKU numbers, and status indicators.

Adding inventory is straightforward through manual entry or bulk upload. For manual entry, sellers input product details including title, description, category, price, quantity available, and product images (minimum 3, maximum 10 images per listing). The bulk upload feature accepts CSV files following Shoplite's template format, enabling hundreds of products to be added or updated simultaneously. This feature automatically validates data and flags errors before finalizing uploads.

Stock level automation helps prevent overselling. When a product sells, inventory automatically decrements by the purchased quantity. Sellers can set minimum stock thresholds that trigger email alerts, allowing timely reordering. The system can also automatically pause listings when inventory reaches zero, preventing orders for out-of-stock items. Integration with third-party inventory management software is available through API connections for sellers managing inventory across multiple platforms.

Inventory reports provide insights into stock movement, best-selling products, slow-moving items, and inventory value. The system tracks inventory turnover rates and suggests pricing adjustments for items with extended shelf time. Sellers can also implement inventory reservations, where stock is temporarily held during the checkout process to prevent simultaneous purchases exceeding available quantity. Seasonal inventory planning tools help sellers forecast demand and optimize stock levels based on historical sales data.
"""
    },
     {
        "id": "doc10",
        "title": "Shoplite Commission and Fee Structure",
        "content": """Understanding Shoplite's fee structure is essential for sellers to price products competitively while maintaining profitability. The platform operates on a commission-based model where fees are deducted from each sale before disbursing funds to sellers. Fee structure varies based on seller subscription tier and product category, ensuring fairness across different business models.

Basic tier sellers pay a flat 12% commission on each sale plus a $0.30 transaction fee per order. Professional tier sellers ($39.99 monthly subscription) benefit from reduced commission rates of 8% plus the same $0.30 transaction fee. Enterprise sellers with annual sales exceeding $500,000 can negotiate custom commission rates, typically ranging from 5-7%. These fees cover payment processing, platform maintenance, customer support, and fraud protection services.

Category-specific commissions apply to certain product types. Electronics and high-value items carry a 10% commission rate (8% for Professional tier) due to higher support and fraud prevention costs. Fashion and apparel maintain standard rates. Books, media, and digital products have reduced commissions of 6% to encourage content diversity. Handmade and artisan products receive preferential 7% rates to support small creators.

Additional fees may apply for optional services. Featured product listings cost $5 per week per product, placing items prominently in category pages. Promoted listings through Shoplite's advertising platform operate on a cost-per-click model with minimum $10 daily budgets. Currency conversion fees of 2.5% apply to international transactions. Return processing fees of $3 per return are charged when customers utilize prepaid return labels. All fees are transparently displayed in the Seller Dashboard, and comprehensive monthly statements detail all charges and commissions for accounting purposes."""
    },
     {
        "id": "doc11",
        "title": "Shoplite Customer Support Procedures",
        "content": """Shoplite provides multiple customer support channels to address user concerns efficiently. Customers can access support through live chat (available 9 AM - 9 PM EST), email support (24/7 response within 24 hours), phone support for urgent issues, and a comprehensive help center with articles covering common questions. The support system uses intelligent routing to direct inquiries to specialized teams based on issue type.

The help center serves as the first resource, containing over 500 articles organized by topics including account management, ordering, payments, shipping, returns, and technical issues. A robust search function helps users find relevant articles quickly. Each article includes step-by-step instructions with screenshots and video tutorials where applicable. The system tracks article helpfulness through user feedback, continuously improving content based on customer needs.

For direct support, customers should first log into their accounts to access personalized assistance. Logged-in users receive faster service as support agents can immediately view order history, account details, and previous interactions. When contacting support, customers should provide their order number, detailed description of the issue, and any relevant screenshots or documentation. This information helps support agents resolve issues efficiently, often within the first interaction.

Support tickets are prioritized based on urgency and issue type. Critical issues like payment failures or account security concerns receive immediate attention, typically resolved within 2-4 hours. Standard inquiries about orders or products are addressed within 24 hours. General questions may take up to 48 hours during peak periods. Customers receive automatic email updates when their support tickets are assigned, updated, or resolved. For complex issues requiring escalation, senior support specialists and department managers become involved. All interactions are documented in the customer's account history for reference in future inquiries."""
    },
     {
        "id": "doc12",
        "title": "Shoplite Mobile App Features",
        "content": """The Shoplite mobile app provides full platform functionality optimized for iOS and Android devices, enabling shopping on the go. Available free from the App Store and Google Play, the app offers a streamlined interface designed for touch navigation and smaller screens. Users can download the app, log in with existing credentials, and access all account features without desktop limitations.

The app's home screen features personalized product recommendations based on browsing history, past purchases, and saved preferences. A bottom navigation bar provides quick access to search, categories, shopping cart, and account settings. The search function includes voice search capability, allowing users to speak product queries instead of typing. Barcode scanning functionality enables users to scan physical product barcodes to find matching items on Shoplite, useful for price comparisons while shopping in physical stores.

Push notifications keep users informed about order updates, delivery status changes, price drops on watched items, and exclusive mobile app deals. Users can customize notification preferences to receive only relevant alerts. The app includes biometric authentication options (fingerprint or face recognition) for quick and secure login, eliminating the need to enter passwords repeatedly.

Mobile-exclusive features include augmented reality product preview for furniture and home decor items, allowing users to visualize products in their space using the device camera. One-touch checkout streamlines the purchase process by storing payment and shipping information for instant transactions. The app's wishlist syncs across devices, and users can share products directly to social media platforms. Offline mode allows browsing previously viewed products and saved items without internet connection. Location-based features show nearby seller locations and estimated delivery times based on current GPS position.
"""
    },
     {
        "id": "doc13",
        "title": "Shoplite API Documentation for Developers",
        "content": """Shoplite offers a comprehensive RESTful API enabling developers to integrate platform functionality into third-party applications, build custom tools, and automate business processes. The API provides programmatic access to product catalogs, order management, inventory updates, customer data, and analytics. API access requires registration through the Developer Portal and obtaining API credentials including an API key and secret.

The API follows RESTful principles with standard HTTP methods (GET, POST, PUT, DELETE) and returns responses in JSON format. Authentication uses OAuth 2.0 protocol, ensuring secure access to platform resources. Developers must include the API key in request headers for all endpoints. Rate limiting applies to prevent abuse, allowing 1000 requests per hour for standard accounts and 5000 requests per hour for premium accounts. Exceeding rate limits results in temporary throttling with clear error messages indicating wait times.

Key API endpoints include product listing management (create, update, delete listings), inventory management (real-time stock updates), order processing (retrieve order details, update order status), customer management (access customer information with proper permissions), and webhook subscriptions (receive real-time notifications for specific events like new orders or inventory changes). Each endpoint includes detailed documentation with request parameters, response formats, and example code in multiple programming languages including Python, JavaScript, PHP, and Ruby.

The API documentation portal provides interactive testing tools allowing developers to make API calls directly from the browser to understand request-response patterns. Sandbox environment availability enables testing without affecting production data. Comprehensive error code documentation helps developers troubleshoot issues quickly. The API maintains backward compatibility, with deprecated endpoints receiving 6-month advance notice before removal. Developer support is available through dedicated forums, email support, and monthly webinars covering API updates and best practices."""
    },
     {
        "id": "doc14",
        "title": "Shoplite Security and Privacy Policies",
        "content": """Security and privacy are foundational principles at Shoplite, reflected in comprehensive policies and technical implementations protecting user data. All data transmissions use 256-bit SSL/TLS encryption, ensuring information remains secure during transfer between users' devices and Shoplite servers. Our infrastructure includes multiple security layers including firewalls, intrusion detection systems, and regular security audits by independent third-party firms.

User data is stored in secure, geographically distributed data centers with redundant backups and disaster recovery protocols. Access to customer data is strictly limited to authorized personnel who require it for legitimate business purposes, and all access is logged and monitored. Employee security training is mandatory, covering data protection principles, phishing awareness, and incident response procedures. Background checks are conducted on all employees with data access privileges.

Privacy policy transparency is central to our operations. We collect only necessary information for platform functionality including account details, purchase history, payment information, shipping addresses, and browsing behavior for personalization. We never sell personal information to third parties. Data sharing occurs only with essential service providers like payment processors and shipping carriers, bound by contractual confidentiality agreements. Users have full control over data sharing preferences through privacy settings in their account dashboard.

Users can exercise data rights including accessing their stored data, requesting data portability in machine-readable formats, correcting inaccurate information, and requesting account deletion. Account deletion requests are processed within 30 days, permanently removing personal data except where legal retention requirements apply (such as tax records and dispute resolution documentation). Two-factor authentication is available for enhanced account security, and we recommend enabling it for all accounts, especially seller accounts handling financial transactions."""
    },
     {
        "id": "doc15",
        "title": "Shoplite Promotional Codes and Discounts",
        "content": """Shoplite's promotional system enables both platform-wide campaigns and seller-specific discounts, providing value to customers while driving sales. Promotional codes are alphanumeric strings entered during checkout to receive discounts. Codes are case-sensitive and must be entered exactly as provided. The system validates codes in real-time, displaying applicable discounts immediately and adjusting order totals before payment.

Several promotional code types exist on the platform. Percentage discounts reduce order total by a specified percentage (e.g., SAVE20 for 20% off). Fixed amount discounts provide specific dollar reductions (e.g., GET10OFF for $10 discount). Free shipping codes waive shipping charges regardless of order value. Buy-one-get-one (BOGO) promotions apply automatically when qualifying items are added to cart. Category-specific codes apply only to products within designated categories.

Promotional codes include usage restrictions and eligibility criteria. Minimum purchase requirements may apply, requiring orders to reach specific thresholds before codes activate. Maximum discount caps limit total savings even for high-value orders. Some codes are single-use only, while others allow multiple applications by the same user. Expiration dates are enforced strictly, with expired codes rejected at checkout. New customer codes require account age verification, available only to users registered within the past 30 days.

Customers can discover promotional codes through multiple channels. Email subscribers receive exclusive codes for platform sales and special events. Social media followers get early access to limited-time promotions. The Shoplite app features a "Deals" section aggregating active promotions and codes. Third-party coupon websites may share codes, though Shoplite doesn't control external listings. Sellers can create custom codes for their products through the Seller Dashboard, setting discount percentages, usage limits, and validity periods. Stacking multiple codes is generally prohibited, with the system applying whichever code provides greater savings."""
    },
     {
        "id": "doc16",
        "title": "Shoplite Seller Performance Metrics and Requirements",
        "content": """Maintaining high seller performance standards is essential for continued access to Shoplite's platform and customer base. The platform monitors seller performance across multiple metrics, displayed in the Seller Dashboard with color-coded indicators showing performance status. Sellers must maintain minimum thresholds across all metrics to remain in good standing and avoid account restrictions.

The Order Defect Rate (ODR) measures negative customer experiences including product not received claims, significantly not as described claims, and credit card chargebacks. Sellers must maintain ODR below 1%, meaning fewer than 1 in 100 orders can result in these issues. High ODR indicates systemic problems with product quality, shipping reliability, or listing accuracy. Consistent ODR violations result in account warnings, selling restrictions, or permanent suspension.

Order fulfillment metrics track shipping performance. Pre-fulfillment cancel rate must stay below 2.5%, as excessive cancellations frustrate customers and harm platform reputation. Late shipment rate should remain under 4%, with orders shipped within the promised handling time frame. Valid tracking rate requires at least 95% of orders to have working tracking numbers uploaded within handling time. These metrics ensure customers receive orders promptly with visibility into shipment status.

Customer service metrics evaluate seller responsiveness and satisfaction. Response time to customer messages must average under 24 hours, demonstrating commitment to addressing customer concerns promptly. Customer satisfaction rating, based on post-purchase surveys, should maintain 4.0 stars or higher on a 5-star scale. Policy compliance violations are tracked separately, with violations for prohibited items, intellectual property infringement, or terms of service breaches resulting in immediate penalties.

Performance reviews occur monthly, with underperforming sellers receiving notifications detailing specific metrics requiring improvement. Sellers have 30 days to address performance issues before escalating consequences. Severe or repeated violations may result in temporary selling privileges suspension or permanent account termination. High-performing sellers with excellent metrics may qualify for benefits including reduced commission rates, featured seller badges, and priority customer support access."""
    },
     {
        "id": "doc17",
        "title": "Shoplite Product Listing Best Practices",
        "content": """Creating effective product listings is crucial for seller success on Shoplite, directly impacting visibility, conversion rates, and customer satisfaction. High-quality listings combine accurate information, compelling visuals, and strategic keyword usage to attract and convert potential buyers. Following best practices ensures listings comply with platform policies while maximizing sales potential.

Product titles should be clear, descriptive, and keyword-rich without being spammy. Include essential information like brand, product type, key features, size, color, and quantity. Titles should be 60-200 characters, balancing descriptiveness with readability. Avoid ALL CAPS, excessive punctuation, or promotional language like "BEST DEAL EVER." Poor example: "Amazing SALE!!! Shoes - Great Price!!!!" Good example: "Nike Air Max 270 Men's Running Shoes, Size 10, Black/White, Breathable Mesh."

Product descriptions require comprehensive details helping customers make informed decisions. Include product dimensions, materials, functionality, compatibility information, and care instructions. Use paragraph breaks and bullet points for readability. Describe benefits, not just features - explain how the product solves problems or improves the customer's life. Maintain honest, accurate descriptions matching the actual product to avoid returns and negative reviews.

Image quality significantly impacts conversion rates. Listings require minimum 3 images, but 7-10 images optimal. Primary image should show the product on white background, meeting Shoplite's guidelines. Additional images should display multiple angles, close-up details, scale references, products in use, and packaging contents. Images must be high-resolution (minimum 1000x1000 pixels), well-lit, and accurately represent product color and condition. Videos increase engagement and can showcase product functionality effectively.

Pricing strategy affects both visibility and sales. Research competitor pricing for similar products to remain competitive. Factor in all costs including product cost, Shoplite commissions, shipping, and desired profit margin. Consider promotional pricing strategies for new listings to build initial reviews and sales velocity. Shipping costs should be transparent and competitive - offering free shipping by building costs into product price often increases conversion rates substantially.

"""
    },
     {
        "id": "doc18",
        "title": "Shoplite Dispute Resolution and Buyer Protection",
        "content": """Shoplite's dispute resolution system protects buyers and sellers when conflicts arise regarding orders. The platform mediates disputes fairly, gathering evidence from both parties before reaching decisions. Understanding the dispute process helps users navigate conflicts effectively and achieve satisfactory resolutions.

Common dispute types include items not received, items significantly different from description, defective or damaged products, and unauthorized transactions. Buyers should first attempt resolution directly with sellers through Shoplite's messaging system, as many issues can be resolved quickly through direct communication. If seller communication fails or sellers are unresponsive for 48 hours, buyers can escalate to formal dispute resolution.

Initiating disputes requires accessing "My Orders," selecting the problematic order, and clicking "Open Dispute." The system guides users through providing dispute details, selecting dispute reason, and uploading supporting evidence like photos, videos, or communication records. Comprehensive evidence strengthens cases significantly. Once submitted, sellers receive notification and have 3 business days to respond with their perspective and counter-evidence.

The dispute review process involves Shoplite's specialized dispute resolution team analyzing all submitted information objectively. Review typically takes 5-7 business days for standard cases, though complex disputes may require longer. The team may request additional information from either party during review. Decisions consider evidence quality, platform policies, seller performance history, and precedent from similar cases.

Possible dispute resolutions include full refunds to buyers, partial refunds reflecting item condition or missing components, return requirements where buyers ship items back for full refunds, or dispute denial if evidence supports seller's position. All resolution decisions are final unless new evidence emerges. Buyer Protection guarantees ensure eligible purchases are protected - if items don't arrive or don't match descriptions, buyers receive refunds. Protection covers purchases up to $2,500 per transaction, providing confidence when shopping on the platform. Repeated frivolous disputes from buyers or consistent dispute losses for sellers result in account reviews and potential restrictions."""
    }
]



In [4]:
# -------------------------------------------------------------------
# Ground Truth Q&A (20 items) from ground-truth-qa.md
# -------------------------------------------------------------------
GROUND_TRUTH_QA = [
    {
        "q_id": "Q01",
        "question": "How long does it take for seller account verification to complete?",
        "expected_context": "Document 8: Shoplite Seller Account Setup and Management",
        "answer": "Seller account verification typically takes 2-3 business days. During this time, the verification team reviews submitted documents including business licenses, tax registration certificates, and government-issued ID to ensure compliance with legal requirements and platform policies. Some sellers may be asked to provide additional documentation during this process.",
        "required_keywords": ["2-3 business days", "verification", "documents", "business"],
        "forbidden_terms": ["instant approval", "same day", "no verification", "automatic approval"]
    },
    {
        "q_id": "Q02",
        "question": "What are the shipping timeframes for standard delivery on Shoplite?",
        "expected_context": "Document 5: Shoplite Order Tracking and Delivery",
        "answer": "Standard shipping on Shoplite typically takes 5-7 business days for delivery. Additionally, expedited shipping takes 2-3 business days, and express overnight shipping delivers within 1 business day for orders placed before 2 PM. International orders have extended delivery windows of 10-21 business days depending on destination country and customs processing.",
        "required_keywords": ["5-7 business days", "standard shipping", "delivery"],
        "forbidden_terms": ["overnight", "same day", "1-2 days for standard shipping"]
    },
    {
        "q_id": "Q03",
        "question": "What is the minimum password requirement for creating a Shoplite account?",
        "expected_context": "Document 1: Shoplite User Registration Process",
        "answer": "Shoplite requires passwords to be a minimum of 8 characters and must include uppercase letters, lowercase letters, numbers and special character. This ensures account security for all users during the registration process.",
        "required_keywords": ["8 characters", "uppercase", "lowercase", "numbers", "special character"],
        "forbidden_terms": ["no requirements", "any password", "6 characters"]
    },
    {
        "q_id": "Q04",
        "question": "What commission rate do Basic tier sellers pay on Shoplite?",
        "expected_context": "Document 10: Shoplite Commission and Fee Structure",
        "answer": "Basic tier sellers pay a flat 12% commission on each sale plus a $0.30 transaction fee per order. Professional tier sellers who pay the $39.99 monthly subscription benefit from reduced commission rates of 8% plus the same $0.30 transaction fee.",
        "required_keywords": ["12%", "commission", "Basic tier", "$0.30", "transaction fee"],
        "forbidden_terms": ["8%", "free", "no commission"]
    },
    {
        "q_id": "Q05",
        "question": "How many images are required minimum for a product listing?",
        "expected_context": "Document 17: Shoplite Product Listing Best Practices",
        "answer": "Product listings require a minimum of 3 images, but 7-10 images are optimal for better conversion rates. The primary image should show the product on a white background, and additional images should display multiple angles, close-up details, scale references, products in use, and packaging contents. Images must be high-resolution with minimum 1000x1000 pixels.",
        "required_keywords": ["3 images", "minimum", "high-resolution", "1000x1000 pixels"],
        "forbidden_terms": ["1 image", "no images required", "500x500"]
    },
    {
        "q_id": "Q06",
        "question": "What is the Order Defect Rate (ODR) threshold that sellers must maintain?",
        "expected_context": "Document 16: Shoplite Seller Performance Metrics and Requirements",
        "answer": "Sellers must maintain an Order Defect Rate (ODR) below 1%, meaning fewer than 1 in 100 orders can result in negative customer experiences including product not received claims, significantly not as described claims, and credit card chargebacks. High ODR indicates systemic problems and can result in account warnings, selling restrictions, or permanent suspension.",
        "required_keywords": ["1%", "ODR", "Order Defect Rate", "below"],
        "forbidden_terms": ["5%", "10%", "no limit", "flexible"]
    },
    {
        "q_id": "Q07",
        "question": "How long do customers have to complete email verification after registration?",
        "expected_context": "Document 1: Shoplite User Registration Process",
        "answer": "Email verification must be completed within 24 hours of registration to activate the account. This is mandatory for all users during the registration process at shoplite.com/register.",
        "required_keywords": ["24 hours", "email verification", "mandatory"],
        "forbidden_terms": ["optional", "no time limit", "7 days", "instant"]
    },
    {
        "q_id": "Q08",
        "question": "What is the rate limit for standard API accounts per hour?",
        "expected_context": "Document 13: Shoplite API Documentation for Developers",
        "answer": "Standard API accounts are allowed 1000 requests per hour, while premium accounts can make 5000 requests per hour. Exceeding rate limits results in temporary throttling with clear error messages indicating wait times. The API uses OAuth 2.0 protocol for authentication.",
        "required_keywords": ["1000 requests", "per hour", "standard", "rate limit"],
        "forbidden_terms": ["unlimited", "5000 for standard accounts", "no limits"]
    },
    {
        "q_id": "Q09",
        "question": "I want to start selling on Shoplite. What's the complete process from registration to being able to list products, and what fees should I expect?",
        "expected_context": "Document 8: Seller Account Setup and Management + Document 10: Commission and Fee Structure",
        "answer": "To become a Shoplite seller, first create a basic account, then navigate to Seller Central to begin the business application. You'll need to provide business information including legal business name, tax identification number, business address, and bank account details. Submit documents like business licenses, tax registration certificates, and government-issued ID. Verification takes 2-3 business days. Once approved, you can access the Seller Dashboard to create product listings. For fees, Basic tier is free but charges 12% commission plus $0.30 per transaction. Professional tier costs $39.99 monthly with reduced 8% commission. Category-specific commissions may apply - electronics are 10%, books/media are 6%, and handmade products are 7%.",
        "required_keywords": ["Seller Central", "business verification", "2-3 business days", "12% commission", "Basic tier", "Professional tier", "$39.99"],
        "forbidden_terms": ["instant approval", "no fees", "free selling"]
    },
    {
        "q_id": "Q10",
        "question": "If I receive a damaged product, what are my options for returns and how long will it take to get my money back?",
        "expected_context": "Document 6: Return and Refund Policies + Document 5: Order Tracking and Delivery",
        "answer": "For damaged products, log into your account, go to 'My Orders,' select the order, and click 'Request Return.' You'll select the damage reason and upload photos of the damage. Return requests are reviewed within 24-48 hours. Since the item is damaged, Shoplite covers return shipping costs and you'll receive a prepaid return shipping label via email. Items must be shipped back within 14 days of return approval. Once the returned item is received and inspected at the facility, refunds are processed within 5-7 business days and credited to your original payment method. The total timeline is about 2-3 weeks depending on shipping time.",
        "required_keywords": ["Request Return", "24-48 hours", "prepaid label", "Shoplite covers", "5-7 business days", "photos"],
        "forbidden_terms": ["immediate refund", "no return needed", "keep the item"]
    },
    {
        "q_id": "Q11",
        "question": "How can I track my order and what notifications will I receive during the shipping process?",
        "expected_context": "Document 5: Order Tracking and Delivery + Document 12: Mobile App Features",
        "answer": "After placing an order, you'll receive immediate email confirmation with a unique order number and estimated delivery timeframe. You can track orders through 'My Orders' in your account or by entering the order number on the tracking page. Updates include order confirmation, payment processing, seller preparation, shipment dispatch, in-transit status, out for delivery, and final delivery confirmation with timestamps. You'll get email and SMS notifications (if enabled), and mobile app users also receive push notifications.",
        "required_keywords": ["My Orders", "real-time updates", "email", "SMS notifications", "order number", "tracking page"],
        "forbidden_terms": ["no tracking", "manual check only", "call seller"]
    },
    {
        "q_id": "Q12",
        "question": "What security measures does Shoplite have in place to protect my payment information and personal data?",
        "expected_context": "Document 4: Payment Methods and Security + Document 14: Security and Privacy Policies",
        "answer": "Shoplite implements 256-bit SSL/TLS encryption for all data transmissions. Payment transactions use 256-bit SSL encryption and Shoplite never stores complete credit card numbers, instead using tokenization. Processing partners are PCI DSS compliant. Infrastructure has firewalls, intrusion detection, and third-party audits. Data is stored in secure distributed centers. Access is limited, logged, and monitored. Two-factor authentication is available and fraud detection systems run continuously.",
        "required_keywords": ["256-bit SSL encryption", "tokenization", "PCI DSS", "two-factor authentication", "fraud detection", "secure data centers"],
        "forbidden_terms": ["no encryption", "stores credit cards", "unsecure"]
    },
    {
        "q_id": "Q13",
        "question": "I'm having issues with my order. What are all the ways I can get help, and how quickly will someone respond?",
        "expected_context": "Document 11: Customer Support Procedures + Document 12: Mobile App Features",
        "answer": "Shoplite offers live chat (9 AM - 9 PM EST), 24/7 email support (within 24 hours), phone support for urgent issues, and a help center with 500+ articles. Log into your account and provide order details for faster service. Critical issues (like payment failures) are handled within 2-4 hours, standard order inquiries within 24 hours, and general questions up to 48 hours during peaks. Mobile app provides push notifications for ticket updates.",
        "required_keywords": ["live chat", "9 AM - 9 PM EST", "email support", "24 hours", "help center", "phone support", "order number"],
        "forbidden_terms": ["no support available", "weeks to respond", "email only"]
    },
    {
        "q_id": "Q14",
        "question": "What's the difference between the shopping cart and 'Save for Later' feature, and how long are items stored?",
        "expected_context": "Document 2: Shopping Cart Features + Document 1: User Registration Process",
        "answer": "The shopping cart holds items for purchase and is preserved across sessions for logged-in users. 'Save for Later' lets you move items out of the active cart. Guest users' carts are stored for 7 days via cookies. Logged-in users have indefinite storage.",
        "required_keywords": ["Save for Later", "cart", "logged-in users", "preserved", "guest users", "7 days", "browser cookies"],
        "forbidden_terms": ["same thing", "no difference", "24 hours only"]
    },
    {
        "q_id": "Q15",
        "question": "How do product reviews work, and how can I tell if a review is from a real buyer?",
        "expected_context": "Document 7: Product Reviews and Ratings + Document 17: Product Listing Best Practices",
        "answer": "Only verified buyers can submit reviews, available 3 days after delivery and up to 90 days. Reviews include 1-5 stars, text, and optional media. Moderation occurs within 24-48 hours. Verified purchase badges indicate authentic buyers. Ratings are averaged, reviews can be sorted, and top reviewers earn badges.",
        "required_keywords": ["verified buyers", "Verified purchase badges", "3 days after delivery", "90 days", "moderation", "1-5 stars"],
        "forbidden_terms": ["anyone can review", "instant posting", "no verification"]
    },
    {
        "q_id": "Q16",
        "question": "I want to offer a discount on my products. How do promotional codes work for sellers, and what types can I create?",
        "expected_context": "Document 15: Promotional Codes and Discounts + Document 8: Seller Account Setup",
        "answer": "Sellers can create codes in the Seller Dashboard: percentage discounts, fixed amounts, free shipping, and BOGO. Restrictions can include min purchase, usage limits, caps, expiration, and eligibility. Codes are entered at checkout and validated real-time. Customers find them in email, app, or social media. Stacking is not allowed; the best discount applies.",
        "required_keywords": ["Seller Dashboard", "percentage discounts", "fixed amount", "free shipping", "usage limits", "validity periods", "minimum purchase"],
        "forbidden_terms": ["automatic discounts only", "cannot create codes", "Shoplite creates all codes"]
    },
    {
        "q_id": "Q17",
        "question": "What happens if I have a dispute with a seller, and what evidence should I provide?",
        "expected_context": "Document 18: Dispute Resolution + Document 11: Customer Support Procedures",
        "answer": "First try resolving with the seller. If no response after 48 hours, open a dispute in 'My Orders.' Provide details, reason, and evidence (photos, videos, chat logs). Seller has 3 business days to respond. Shoplite resolves in 5-7 business days. Buyer Protection covers up to $2,500.",
        "required_keywords": ["48 hours", "Open Dispute", "photos", "videos", "evidence", "3 business days", "5-7 business days", "Buyer Protection", "$2,500"],
        "forbidden_terms": ["instant resolution", "automatic refund", "no evidence needed"]
    },
    {
        "q_id": "Q18",
        "question": "As a new seller, what performance standards do I need to maintain to avoid account issues?",
        "expected_context": "Document 16: Seller Performance Metrics + Document 8: Seller Account Setup",
        "answer": "Key standards: ODR <1%, cancel rate <2.5%, late shipment <4%, valid tracking ≥95%, respond within 24 hours, maintain ≥4.0 stars. Reviews occur monthly. Failure may result in suspension, while high performers get benefits.",
        "required_keywords": ["1%", "ODR", "2.5%", "4%", "95%", "24 hours", "4.0 stars", "monthly reviews"],
        "forbidden_terms": ["no requirements", "flexible standards", "warnings only"]
    },
    {
        "q_id": "Q19",
        "question": "I want to use Shoplite's API to integrate with my inventory system. What do I need to get started and what can I do with it?",
        "expected_context": "Document 13: API Documentation + Document 9: Inventory Management",
        "answer": "Register in Developer Portal for API key and secret. RESTful API with OAuth 2.0. Standard = 1000 requests/hour, premium = 5000. Endpoints: listings, inventory, orders, customer data, webhooks. Manage stock, alerts, automation. Sandbox and support available.",
        "required_keywords": ["Developer Portal", "API key", "OAuth 2.0", "1000 requests", "REST", "inventory management", "real-time", "webhook"],
        "forbidden_terms": ["no authentication", "unlimited requests", "automatic access"]
    },
    {
        "q_id": "Q20",
        "question": "What are all the features available in the Shoplite mobile app that I can't get on the website?",
        "expected_context": "Document 12: Mobile App Features + Document 2: Cart Features + Document 5: Order Tracking",
        "answer": "The app has voice search, barcode scanning, push notifications, biometric login, augmented reality previews, one-touch checkout, offline browsing, and location-based delivery estimates. Wishlist syncs and social sharing are also app-specific.",
        "required_keywords": ["voice search", "barcode scanning", "push notifications", "biometric authentication", "augmented reality", "one-touch checkout", "offline mode", "location-based"],
        "forbidden_terms": ["no different features", "same as website", "nothing special"]
    }
]

print(f"Loaded {len(GROUND_TRUTH_QA)} Q&A items")


Loaded 20 Q&A items


In [18]:
PROMPTS_YAML = """
    # RAG Assistant Prompts Configuration
version: "1.0"
created: "2025-09-23"
author: "Karl Sassine"
description: "Structured prompts for Shoplite customer service RAG assistant"

# =============================================================================
# BASE RETRIEVAL PROMPT - Standard customer service interactions
# =============================================================================
base_retrieval_prompt:
  role: |
    You are a helpful and knowledgeable Shoplite customer service assistant with expertise in e-commerce operations, policies, and technical support.

  goal: |
    Provide accurate, helpful answers to customer questions using ONLY the information from the provided Shoplite documentation context. Your primary objective is to resolve customer inquiries efficiently while maintaining high accuracy and customer satisfaction.

  context_guidelines:
    - "Use ONLY information explicitly stated in the provided document snippets"
    - "Cite specific document titles when referencing information"
    - "If multiple documents contain relevant information, synthesize them coherently"
    - "Maintain factual accuracy - never infer or assume information not in the context"
    - "If context is insufficient, acknowledge limitations clearly"

  constraints:
    - "Do NOT make up information not present in the retrieved documents"
    - "Do NOT provide information from general knowledge about e-commerce"
    - "Do NOT speculate about policies or procedures not explicitly documented"
    - "Do NOT provide specific legal, financial, or medical advice"
    - "Do NOT share information about competitors or compare with other platforms"

  tone:
    style: "Professional, friendly, and empathetic"
    characteristics:
      - "Conversational yet informative"
      - "Patient and understanding of customer frustrations"
      - "Confident in provided information"
      - "Apologetic when unable to help"
      - "Encouraging and supportive"

  input: |
    {{input text}}

  response_format: |
    **Answer:** [Provide clear, direct answer based on retrieved context]

    **Sources:** [List document titles referenced, e.g., "Document 5: Order Tracking and Delivery"]

    **Additional Help:** [If applicable, suggest related topics or next steps]

  quality_checks:
    - "Verify all facts match the source documents exactly"
    - "Ensure response directly addresses the user's question"
    - "Check that all cited sources are actually used in the answer"
    - "Confirm tone is appropriate for the query type"

# =============================================================================
# COMPLEX MULTI-DOCUMENT PROMPT - For questions requiring multiple sources
# =============================================================================
complex_multi_document_prompt:
  role: |
    You are a senior Shoplite customer service specialist skilled at synthesizing information from multiple documentation sources to provide comprehensive answers.

  goal: |
    Address complex customer questions that span multiple topics or require information from several documentation sources. Provide complete, well-organized answers that connect related information logically while maintaining accuracy.

  context_guidelines:
    - "Identify information from all relevant retrieved documents"
    - "Synthesize information coherently, showing relationships between topics"
    - "Organize answer logically with clear transitions between document sources"
    - "Prioritize the most relevant information for the specific question"
    - "Cite each document used and indicate which information comes from where"

  constraints:
    - "Do NOT contradict information from different documents"
    - "Do NOT combine information in ways that create new meanings"
    - "Do NOT omit critical details from any relevant source"
    - "Do NOT overwhelm with unnecessary information from tangentially related docs"
    - "Maintain clear boundaries between facts from different documents"

  tone:
    style: "Thorough, organized, and consultative"
    characteristics:
      - "Comprehensive without being overwhelming"
      - "Clear and well-structured"
      - "Professional and authoritative"
      - "Patient with complex topics"
      - "Helpful in connecting related information"

  input: |
    {{input text}}

  response_format: |
    **Complete Answer:**
    [Provide organized response addressing all aspects of the question]

    [For multi-part questions, use clear sections:]

    **Part 1: [Topic]**
    [Information from relevant documents]

    **Part 2: [Topic]**
    [Information from relevant documents]

    **Summary:** [Brief recap of key points if answer is long]

    **Sources Used:**
    - [Document 1 title] - [What information it provided]
    - [Document 2 title] - [What information it provided]

    **Related Information:** [Optional: mention related topics customer might want to know]

  quality_checks:
    - "Verify answer addresses all parts of the question"
    - "Ensure smooth flow between information from different sources"
    - "Check that synthesis doesn't create false connections"
    - "Confirm all critical information is included"
    - "Validate that organization makes answer easy to understand"

# =============================================================================
# INSUFFICIENT CONTEXT PROMPT - When retrieved docs don't answer the question
# =============================================================================
insufficient_context_prompt:
  role: |
    You are an honest and transparent Shoplite customer service assistant who prioritizes accuracy over appearing knowledgeable.

  goal: |
    Clearly communicate when the available documentation does not contain information needed to answer the customer's question, while still being helpful by suggesting alternative resources or actions.

  context_guidelines:
    - "Carefully review all retrieved documents to confirm information is truly absent"
    - "Do NOT attempt to answer using general knowledge or assumptions"
    - "Explain specifically what information is missing"
    - "Suggest appropriate next steps or resources"

  constraints:
    - "NEVER make up information to fill gaps"
    - "NEVER guess at policies or procedures"
    - "Do NOT apologize excessively - be direct and helpful"
    - "Do NOT suggest the question is invalid - focus on documentation limitations"

  tone:
    style: "Honest, helpful, and solution-oriented"
    characteristics:
      - "Direct and transparent about limitations"
      - "Apologetic but not overly so"
      - "Proactive in suggesting alternatives"
      - "Maintaining customer confidence despite inability to help directly"

  input: |
    {{input text}}

  response_format: |
    **I need to be transparent:** The documentation I have access to doesn't contain specific information about [topic of question].

    **What I searched:** [Briefly mention what documents were checked]

    **How I can help:**
    - [Suggest contacting support channels: live chat, email, phone]
    - [Mention if related information is available on a different topic]
    - [Provide relevant support contact details if applicable]

    **Next Steps:** [Clear action items for the customer]

  quality_checks:
    - "Confirm that information truly isn't in any retrieved document"
    - "Verify suggested alternatives are appropriate"
    - "Ensure response is helpful despite not answering directly"
    - "Check that tone balances honesty with maintaining customer trust"

# =============================================================================
# CLARIFICATION REQUEST PROMPT - When questions are ambiguous or unclear
# =============================================================================
clarification_request_prompt:
  role: |
    You are a perceptive Shoplite customer service assistant skilled at identifying ambiguities and asking clarifying questions to better serve customers.

  goal: |
    Recognize when a customer's question is too vague, ambiguous, or could have multiple interpretations. Ask targeted clarifying questions to understand their true need before providing a definitive answer.

  context_guidelines:
    - "Identify specific ambiguities in the customer's question"
    - "Consider multiple possible interpretations"
    - "Review retrieved documents to see if they cover multiple scenarios"
    - "Provide partial information if some aspects are clear"

  constraints:
    - "Do NOT ask for clarification if the question is reasonably clear"
    - "Do NOT ask multiple questions at once - prioritize the most important"
    - "Do NOT make customers feel their question was poorly asked"
    - "Avoid overly technical language in clarification requests"

  tone:
    style: "Helpful, patient, and collaborative"
    characteristics:
      - "Gentle and non-judgmental"
      - "Focused on helping, not criticizing"
      - "Conversational and friendly"
      - "Making the customer feel heard"

  input: |
    {{input text}}

  response_format: |
    **I want to make sure I give you the right information.** [Acknowledge their question]

    **What I understood:** [Restate what's clear from their question]

    **To help you better, could you clarify:** [Ask specific, targeted question]

    [If applicable]
    **In the meantime, here's some general information:** [Provide what you can based on what is clear]

    **Sources:** [If partial information provided]

  quality_checks:
    - "Verify ambiguity genuinely exists"
    - "Ensure clarifying question is specific and helpful"
    - "Check that tone is encouraging, not dismissive"
    - "Confirm any partial information provided is accurate"

# =============================================================================
# POLICY EXPLANATION PROMPT - For detailed policy and procedure questions
# =============================================================================
policy_explanation_prompt:
  role: |
    You are a Shoplite policy expert who explains complex rules, procedures, and requirements in clear, accessible language.

  goal: |
    Help customers understand Shoplite policies, requirements, and procedures by breaking down complex information into digestible explanations while maintaining complete accuracy.

  context_guidelines:
    - "Extract all relevant policy details from documentation"
    - "Identify key requirements, timelines, and conditions"
    - "Note any exceptions or special cases mentioned"
    - "Preserve exact numbers, timeframes, and percentages"
    - "Maintain the exact meaning while simplifying language"

  constraints:
    - "Do NOT simplify to the point of changing meaning"
    - "Do NOT omit important conditions or exceptions"
    - "Do NOT interpret policies - state them as written"
    - "Preserve all numeric values exactly (fees, percentages, timeframes)"
    - "Do NOT suggest policy workarounds or exceptions not mentioned"

  tone:
    style: "Clear, educational, and authoritative"
    characteristics:
      - "Breaking complex information into steps"
      - "Using examples when helpful"
      - "Patient and thorough"
      - "Confident in explanations"
      - "Neutral about policies (not defensive or apologetic)"

  input: |
    {{input text}}

  response_format: |
    **Policy Overview:** [High-level summary of what the policy covers]

    **Key Requirements:**
    - [Requirement 1 with specific details]
    - [Requirement 2 with specific details]
    - [Continue as needed]

    **Important Details:**
    - Timeframes: [Specific timelines]
    - Costs/Fees: [Exact amounts]
    - Conditions: [Any conditions or limitations]

    **Example:** [If helpful, provide a concrete example]

    **Exceptions:** [Note any exceptions mentioned in documentation]

    **Source:** [Document reference]

  quality_checks:
    - "Verify all numbers and timeframes are exact"
    - "Ensure no requirements or conditions are omitted"
    - "Check that simplification doesn't alter meaning"
    - "Confirm examples align with stated policy"
    - "Validate that exceptions are accurately represented"

# =============================================================================
# TECHNICAL ASSISTANCE PROMPT - For API, integration, and technical questions
# =============================================================================
technical_assistance_prompt:
  role: |
    You are a Shoplite technical support specialist with expertise in APIs, integrations, and technical platform features.

  goal: |
    Provide accurate technical guidance for developers, sellers using technical features, and users with integration questions. Focus on being precise with technical details while remaining accessible.

  context_guidelines:
    - "Extract exact technical specifications (endpoints, parameters, limits)"
    - "Note authentication requirements and security protocols"
    - "Identify rate limits, restrictions, and requirements"
    - "Preserve technical terminology accuracy"
    - "Include version information when specified"

  constraints:
    - "Do NOT provide code examples not in the documentation"
    - "Do NOT suggest unofficial integrations or workarounds"
    - "Preserve exact API endpoint names and parameter names"
    - "Do NOT approximate technical limits - provide exact numbers"
    - "Do NOT recommend third-party tools unless mentioned in docs"

  tone:
    style: "Technical, precise, and helpful"
    characteristics:
      - "Using appropriate technical terminology"
      - "Being specific and detail-oriented"
      - "Professional and straightforward"
      - "Assuming technical competence while being clear"
      - "Practical and implementation-focused"

  input: |
    {{input text}}

  response_format: |
    **Technical Overview:** [Brief explanation of the technical aspect]

    **Requirements:**
    - [List prerequisites, credentials, permissions needed]

    **Specifications:**
    - [Technical details: endpoints, formats, protocols, limits]

    **Implementation Notes:**
    - [Key points for implementation]
    - [Rate limits, restrictions]
    - [Authentication details]

    **Resources:**
    - Documentation: [Specific section or document]
    - Testing: [Sandbox or testing tools if mentioned]
    - Support: [Technical support channels]

    **Source:** [Document reference]

  quality_checks:
    - "Verify all technical specifications are exact"
    - "Ensure endpoint names and parameters are correct"
    - "Check that limits and restrictions are accurately stated"
    - "Confirm authentication details are complete"
    - "Validate that no unofficial information is included"

# =============================================================================
# TROUBLESHOOTING PROMPT - For problem-solving and issue resolution
# =============================================================================
troubleshooting_prompt:
  role: |
    You are a Shoplite troubleshooting specialist who helps customers diagnose and resolve issues systematically.

  goal: |
    Guide customers through resolving issues by providing step-by-step troubleshooting based on documented procedures. When documentation provides solutions, walk through them clearly. When it doesn't, direct to appropriate support channels.

  context_guidelines:
    - "Look for troubleshooting steps in retrieved documents"
    - "Identify relevant procedures and requirements that might be causing issues"
    - "Note support channels and escalation paths mentioned"
    - "Consider common issues addressed in documentation"

  constraints:
    - "Do NOT diagnose issues not covered in documentation"
    - "Do NOT promise resolution timeframes not specified"
    - "Do NOT provide workarounds not mentioned in official docs"
    - "Do NOT blame sellers, customers, or the platform"
    - "Stick to documented solutions and support channels"

  tone:
    style: "Supportive, methodical, and solution-focused"
    characteristics:
      - "Empathetic to customer frustration"
      - "Calm and reassuring"
      - "Systematic and organized"
      - "Patient with complex issues"
      - "Optimistic about finding solutions"

  input: |
    {{input text}}

  response_format: |
    **I understand the issue:** [Acknowledge and restate the problem]

    **Let's try these steps:**
    1. [Step 1 based on documentation]
    2. [Step 2 based on documentation]
    3. [Continue as needed]

    **If that doesn't resolve it:**
    [Provide escalation path or support contact information]

    **What to have ready:** [Information they'll need for support]
    - [Order number, screenshots, etc.]

    **Expected timeline:** [If mentioned in docs]

    **Source:** [Document reference]

  quality_checks:
    - "Verify steps come from documentation"
    - "Ensure steps are in logical order"
    - "Check that escalation path is appropriate"
    - "Confirm empathy is balanced with solution focus"
    - "Validate that expectations are accurately set"

# =============================================================================
# SELLER-SPECIFIC PROMPT - For seller account and business questions
# =============================================================================
seller_specific_prompt:
  role: |
    You are a Shoplite seller support specialist focused on helping business users succeed on the platform.

  goal: |
    Assist sellers with account setup, inventory management, performance metrics, fees, policies, and tools. Provide business-focused guidance that helps sellers operate effectively and profitably.

  context_guidelines:
    - "Focus on seller-relevant documentation"
    - "Include business implications (costs, requirements, timelines)"
    - "Note performance requirements and standards"
    - "Highlight tools and features available to sellers"
    - "Extract fee structures and commission details accurately"

  constraints:
    - "Do NOT provide business strategy advice beyond documented features"
    - "Do NOT guarantee sales success or revenue amounts"
    - "Preserve exact fee percentages and costs"
    - "Do NOT suggest policy exceptions for business users"
    - "Do NOT compare Shoplite with competitor platforms"

  tone:
    style: "Professional, business-focused, and supportive"
    characteristics:
      - "Respecting sellers as business partners"
      - "Being direct about requirements and costs"
      - "Practical and efficiency-focused"
      - "Understanding business pressures"
      - "Encouraging but realistic"

  input: |
    {{input text}}

  response_format: |
    **Business Overview:** [High-level answer to their question]

    **Requirements/Process:**
    - [Step-by-step or list of requirements]

    **Costs & Fees:** [If applicable - exact amounts]
    - [Break down fee structure clearly]

    **Performance Standards:** [If applicable - exact metrics]
    - [What they need to maintain]

    **Timeline:** [How long processes take]

    **Tools Available:** [Features/tools that help with their goal]

    **Best Practices:** [If documented]

    **Source:** [Document reference]

  quality_checks:
    - "Verify all fees and costs are exact"
    - "Ensure performance metrics are accurate"
    - "Check that timelines match documentation"
    - "Confirm requirements are complete"
    - "Validate business focus is appropriate"

# =============================================================================
# GENERAL CONFIGURATION
# =============================================================================
general_settings:
  max_response_length: 500  # Maximum words in response
  citation_required: true
  allow_partial_answers: true  # Can provide partial info if full answer unavailable
  default_fallback: "insufficient_context_prompt"

  response_enhancement:
    use_examples: true  # Use examples when helpful
    provide_next_steps: true  # Suggest what customer should do next
    offer_related_info: false  # Don't overwhelm with related topics unless asked

  quality_standards:
    - "All numeric values must be exact (no approximations)"
    - "All timeframes must match documentation exactly"
    - "All document citations must be accurate"
    - "Tone must match prompt specification"
    - "Response must directly address the question"

# =============================================================================
# PROMPT SELECTION LOGIC
# =============================================================================
prompt_selection_guide:
  description: "Guide for selecting appropriate prompt based on query characteristics"

  use_base_retrieval_prompt:
    - "Simple, straightforward questions"
    - "Single-topic queries"
    - "Clear questions with obvious answers"

  use_complex_multi_document_prompt:
    - "Questions with multiple parts"
    - "Queries spanning several topics"
    - "Process questions requiring multiple documentation sources"
    - "Questions asking 'how do I do X and Y'"

  use_insufficient_context_prompt:
    - "Retrieved documents don't contain answer"
    - "Question is out of scope for documentation"
    - "Asking for information not in knowledge base"

  use_clarification_request_prompt:
    - "Question is ambiguous or vague"
    - "Multiple interpretations possible"
    - "Missing critical information to answer"
    - "Question uses unclear pronouns or references"

  use_policy_explanation_prompt:
    - "Questions about rules, requirements, policies"
    - "Asking about fees, commissions, costs"
    - "Return policy, refund policy questions"
    - "Performance metrics and standards"

  use_technical_assistance_prompt:
    - "API questions"
    - "Integration questions"
    - "Developer-focused queries"
    - "Technical specification requests"

  use_troubleshooting_prompt:
    - "Problem or issue being reported"
    - "Something not working questions"
    - "Error or failure scenarios"
    - "Questions starting with 'My [X] isn't working'"

  use_seller_specific_prompt:
    - "Questions from seller perspective"
    - "Business operation questions"
    - "Seller dashboard and tools"
    - "Inventory and order management for sellers"
"""
PROMPTS = yaml.safe_load(PROMPTS_YAML)
print("Loaded prompts:", list(PROMPTS.keys()))

Loaded prompts: ['version', 'created', 'author', 'description', 'base_retrieval_prompt', 'complex_multi_document_prompt', 'insufficient_context_prompt', 'clarification_request_prompt', 'policy_explanation_prompt', 'technical_assistance_prompt', 'troubleshooting_prompt', 'seller_specific_prompt', 'general_settings', 'prompt_selection_guide']


In [21]:
print("Qwen/Qwen1.5-7B-Chat")

model_name = "Qwen/Qwen1.5-7B-Chat"

# Configure 4-bit quantization for Colab GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Model loaded successfully!")
print(f"Model device: {model.device}")

Qwen/Qwen1.5-7B-Chat


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Model loaded successfully!
Model device: cuda:0


In [11]:
EMB_MODEL_NAME = 'all-MiniLM-L6-v2' # small, fast SBERT model
print('Loading embedding model:', EMB_MODEL_NAME)
embed_model = SentenceTransformer(EMB_MODEL_NAME)


# Create corpus and map ids
corpus = [d['content'] for d in KNOWLEDGE_BASE]
doc_ids = [d['id'] for d in KNOWLEDGE_BASE]


# Compute embeddings
print('Computing embeddings for', len(corpus), 'documents...')
embeddings = embed_model.encode(corpus, convert_to_numpy=True)


# Build FAISS index
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d) # inner product for cosine if pre-normalized


# normalize embeddings for cosine similarity
faiss.normalize_L2(embeddings)
index.add(embeddings)
print('FAISS index built with', index.ntotal, 'vectors')


# Helper: id -> doc mapping
DOC_MAP = {d['id']: d for d in KNOWLEDGE_BASE}

Loading embedding model: all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Computing embeddings for 18 documents...
FAISS index built with 18 vectors


In [12]:
print("Loading embedding model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Embedding model loaded successfully!")


Loading embedding model...
Embedding model loaded successfully!


In [13]:
def create_embeddings(documents):
    """Create embeddings for all documents"""
    texts = [doc['content'] for doc in documents]
    embeddings = embedding_model.encode(texts, show_progress_bar=True)
    return embeddings

def build_faiss_index(embeddings):
    """Build FAISS index for similarity search"""
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings.astype('float32'))
    return index

def retrieve_docs(query, index, documents, top_k=3):
    """Retrieve top-k most relevant documents"""
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(query_embedding.astype('float32'), top_k)

    retrieved_docs = []
    for idx in indices[0]:
        retrieved_docs.append(documents[idx])

    return retrieved_docs

def format_prompt(query, retrieved_docs, prompt_config):
    """Format the prompt with retrieved context"""
    context = "\n\n".join([
        f"Document: {doc['title']}\nContent: {doc['content']}"
        for doc in retrieved_docs
    ])

    prompt = f"""{prompt_config['role']}

{prompt_config['goal']}

Guidelines:
{chr(10).join('- ' + g for g in prompt_config['context_guidelines'])}

Context Documents:
{context}

Question: {query}

{prompt_config['response_format']}
"""
    return prompt

def generate_response(prompt, max_length=512):
    """Generate response using the LLM"""
    messages = [
        {"role": "user", "content": prompt}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_length,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    return response.strip()

print("RAG pipeline functions defined successfully!")

RAG pipeline functions defined successfully!


In [14]:
print("Creating embeddings for knowledge base...")
document_embeddings = create_embeddings(KNOWLEDGE_BASE)
print(f"Created embeddings with shape: {document_embeddings.shape}")

print("Building FAISS index...")
faiss_index = build_faiss_index(document_embeddings)
print(f"FAISS index built with {faiss_index.ntotal} documents")

print("\nRAG system ready!")

Creating embeddings for knowledge base...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Created embeddings with shape: (18, 384)
Building FAISS index...
FAISS index built with 18 documents

RAG system ready!


In [27]:
app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint"""
    return jsonify({
        "status": "healthy",
        "model": "Qwen1.5-7B-Chat",
        "documents": len(KNOWLEDGE_BASE)
    })

@app.route('/ping', methods=['POST'])
def ping():
    """Direct LLM interaction without RAG"""
    try:
        data = request.json
        query = data.get('query', '')

        if not query:
            return jsonify({"error": "No query provided"}), 400

        # Generate response without RAG
        messages = [{"role": "user", "content": query}]
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        outputs = model.generate(
            input_ids,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

        response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

        return jsonify({
            "response": response.strip(),
            "mode": "direct"
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/chat', methods=['POST'])
def chat():
    """RAG-enabled chat endpoint"""
    try:
        data = request.json
        query = data.get('query', '')
        top_k = data.get('top_k', 3)

        if not query:
            return jsonify({"error": "No query provided"}), 400


        # Retrieve relevant documents
        retrieved_docs = retrieve_docs(query, faiss_index, KNOWLEDGE_BASE, top_k)

        # Format prompt with context
        prompt = format_prompt(query, retrieved_docs, PROMPTS['base_retrieval_prompt'])

        # Generate response
        response = generate_response(prompt)

        return jsonify({
            "response": response,
            "retrieved_documents": [doc['title'] for doc in retrieved_docs],
            "mode": "rag"
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

print("Flask API configured successfully!")

Flask API configured successfully!


In [23]:
print("="*60)
print("ngrok Token Required")
print("="*60)

ngrok_token = input("Enter your ngrok token: ").strip()

if not ngrok_token:
    print("ERROR: No token provided. Cannot create tunnel.")
else:
    ngrok.set_auth_token(ngrok_token)

    def run_flask():
        app.run(port=5000, use_reloader=False)  # Changed port

    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()
    time.sleep(3)  # Give Flask time to start

    public_url = ngrok.connect(5000)  # Changed port

    print("\n" + "="*60)
    print("🚀 RAG System Deployed Successfully!")
    print("="*60)
    print(f"Public URL: {public_url.public_url}")  # Fixed
    print("\nAvailable Endpoints:")
    print(f"  - Health Check: {public_url.public_url}/health")
    print(f"  - Chat (RAG):   {public_url.public_url}/chat")
    print(f"  - Ping (Direct): {public_url.public_url}/ping")
    print("="*60)

ngrok Token Required
Enter your ngrok token: 33VxNXG9npKPsU3GxeZtaksyS6n_56DPFXofVYq2Cw2m9FJER
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.



🚀 RAG System Deployed Successfully!
Public URL: https://exculpatory-requitable-ahmad.ngrok-free.dev

Available Endpoints:
  - Health Check: https://exculpatory-requitable-ahmad.ngrok-free.dev/health
  - Chat (RAG):   https://exculpatory-requitable-ahmad.ngrok-free.dev/chat
  - Ping (Direct): https://exculpatory-requitable-ahmad.ngrok-free.dev/ping


In [22]:
import requests
import time

print("Testing API endpoints...\n")
time.sleep(2)

base_url = public_url.public_url

print(f"Using base URL: {base_url}\n")

# Test 1: Health check
print("Test 1: Health Check")
try:
    response = requests.get(f"{base_url}/health")
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
except Exception as e:
    print(f"Error: {e}")
print()

# Test 2: RAG chat
print("Test 2: RAG Chat")
test_query = "How do I create a seller account?"
try:
    response = requests.post(
        f"{base_url}/chat",
        json={"query": test_query, "top_k": 3}
    )
    print(f"Query: {test_query}")
    print(f"Status: {response.status_code}")
    result = response.json()
    print(f"Response: {result.get('response', 'N/A')[:200]}...")  # Truncate
    print(f"Retrieved Docs: {result.get('retrieved_documents', [])}")
except Exception as e:
    print(f"Error: {e}")
print()

# Test 3: Direct ping
print("Test 3: Direct Ping (No RAG)")
try:
    response = requests.post(
        f"{base_url}/ping",
        json={"query": "What is 2+2?"}
    )
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
except Exception as e:
    print(f"Error: {e}")
print()

print("✅ All tests completed!")

Testing API endpoints...



INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 22:40:19] "GET /health HTTP/1.1" 200 -


Using base URL: https://exculpatory-requitable-ahmad.ngrok-free.dev

Test 1: Health Check
Status: 200
Response: {'documents': 18, 'model': 'Qwen3Guard-Gen-8B', 'status': 'healthy'}

Test 2: RAG Chat


INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 22:41:12] "POST /chat HTTP/1.1" 200 -


Query: How do I create a seller account?
Status: 200
Response: To create a seller account on Shoplite, follow these steps:

1. Go to the Shoplite website: Visit <https://shoplite.com/register>.
2. Fill in basic information: Provide a valid email address, create a...
Retrieved Docs: ['Shoplite Seller Account Setup and Management', 'Shoplite User Registration Process', 'Shoplite Commission and Fee Structure']

Test 3: Direct Ping (No RAG)


INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 22:41:16] "POST /ping HTTP/1.1" 200 -


Status: 200
Response: {'mode': 'direct', 'response': '2+2 is 4. This is a basic arithmetic operation where two numbers, in this case 2 and 2, are added together to get the total of 4.'}

✅ All tests completed!
